# ML Dataset stats

I want to perform some verification on the ml dataset I generated.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import dask.distributed
import logging
import pathlib
import xarray as xr

from crims2s.dask import create_dask_cluster

In [3]:
ML_DATASET_DIR = '***BASEDIR***/mlready/2021-07-25-test/'

## Boot dask cluster

In [4]:
cluster = create_dask_cluster(cores=6)

In [5]:
cluster

In [6]:
cluster.scale(jobs=2)

In [7]:
client = dask.distributed.Client(cluster)

In [8]:
client

Client Scheduler: tcp://10.20.40.95:45141 Dashboard: http://10.20.40.95:8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


## Load dataset

In [9]:
dataset_files = sorted([x for x in pathlib.Path(ML_DATASET_DIR).iterdir() if '0102.nc' in x.name])

In [10]:
dataset_files[:10]

[PosixPath('***BASEDIR***/mlready/2021-07-25-test/train_example_20000102.nc'),
 PosixPath('***BASEDIR***/mlready/2021-07-25-test/train_example_20010102.nc'),
 PosixPath('***BASEDIR***/mlready/2021-07-25-test/train_example_20020102.nc'),
 PosixPath('***BASEDIR***/mlready/2021-07-25-test/train_example_20030102.nc'),
 PosixPath('***BASEDIR***/mlready/2021-07-25-test/train_example_20040102.nc'),
 PosixPath('***BASEDIR***/mlready/2021-07-25-test/train_example_20050102.nc'),
 PosixPath('***BASEDIR***/mlready/2021-07-25-test/train_example_20060102.nc'),
 PosixPath('***BASEDIR***/mlready/2021-07-25-test/train_example_20070102.nc'),
 PosixPath('***BASEDIR***/mlready/2021-07-25-test/train_example_20080102.nc'),
 PosixPath('***BASEDIR***/mlready/2021-07-25-test/train_example_20090102.nc')]

In [11]:
def preprocess_one_example(dataset):
    return dataset.expand_dims('forecast_time')

In [12]:
features = xr.open_mfdataset(dataset_files[:3], group='/x', concat_dim='forecast_time', preprocess=preprocess_one_example)

In [13]:
model = xr.open_mfdataset(dataset_files[:3], group='/model', concat_dim='forecast_time', preprocess=preprocess_one_example)

In [14]:
obs = xr.open_mfdataset(dataset_files[:3], group='/obs', concat_dim='forecast_time', preprocess=preprocess_one_example)

In [15]:
model

<xarray.Dataset>
Dimensions:             (forecast_time: 3, latitude: 121, lead_time: 46, longitude: 240, realization: 11)
Coordinates:
  * lead_time           (lead_time) timedelta64[ns] 1 days 2 days ... 46 days
  * realization         (realization) int64 0 1 2 3 4 5 6 7 8 9 10
  * forecast_time       (forecast_time) datetime64[ns] 2000-01-02 ... 2002-01-02
  * latitude            (latitude) float64 90.0 88.5 87.0 ... -87.0 -88.5 -90.0
  * longitude           (longitude) float64 0.0 1.5 3.0 ... 355.5 357.0 358.5
    valid_time          (forecast_time, lead_time) datetime64[ns] dask.array<chunksize=(1, 46), meta=np.ndarray>
    forecast_dayofyear  int64 2
    forecast_year       (forecast_time) int64 2000 2001 2002
Data variables:
    t2m                 (forecast_time, lead_time, latitude, longitude, realization) float32 dask.array<chunksize=(1, 46, 121, 240, 11), meta=np.ndarray>
    tp                  (forecast_time, lead_time, latitude, longitude, realization) float32 dask.array<chunksize=(1, 46, 121, 240, 11), meta=np.ndarray>
Attributes:
    GRIB_edition:            2
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2021-05-10T16:32 GRIB to CDM+CF via cfgrib-0.9.9...

In [21]:
obs.isnull().sum().compute()

<xarray.Dataset>
Dimensions:             ()
Coordinates:
    forecast_dayofyear  int64 2
Data variables:
    t2m                 int64 3029970
    pr                  int64 2697210

In [17]:
features

<xarray.Dataset>
Dimensions:             (forecast_time: 3, latitude: 121, lead_time: 46, longitude: 240, realization: 11, variable: 13)
Coordinates:
  * lead_time           (lead_time) timedelta64[ns] 1 days 2 days ... 46 days
  * realization         (realization) int64 0 1 2 3 4 5 6 7 8 9 10
  * forecast_time       (forecast_time) datetime64[ns] 2000-01-02 ... 2002-01-02
  * latitude            (latitude) float64 90.0 88.5 87.0 ... -87.0 -88.5 -90.0
  * longitude           (longitude) float64 0.0 1.5 3.0 ... 355.5 357.0 358.5
    valid_time          (forecast_time, lead_time) datetime64[ns] dask.array<chunksize=(1, 46), meta=np.ndarray>
    forecast_dayofyear  int64 2
    forecast_year       (forecast_time) int64 2000 2001 2002
  * variable            (variable) object 'lsm' 'msl' 'siconc' ... 'v200' 'v850'
Data variables:
    x                   (forecast_time, lead_time, latitude, longitude, realization, variable) float32 dask.array<chunksize=(1, 46, 121, 240, 11, 13), meta=np.ndarray>

In [18]:
obs

<xarray.Dataset>
Dimensions:             (forecast_time: 3, latitude: 121, lead_time: 46, longitude: 240)
Coordinates:
    time                (forecast_time, lead_time) datetime64[ns] dask.array<chunksize=(1, 46), meta=np.ndarray>
  * latitude            (latitude) float64 90.0 88.5 87.0 ... -87.0 -88.5 -90.0
  * longitude           (longitude) float64 0.0 1.5 3.0 ... 355.5 357.0 358.5
  * lead_time           (lead_time) timedelta64[ns] 1 days 2 days ... 46 days
  * forecast_time       (forecast_time) datetime64[ns] 2000-01-02 ... 2002-01-02
    valid_time          (forecast_time, lead_time) datetime64[ns] dask.array<chunksize=(1, 46), meta=np.ndarray>
    forecast_dayofyear  int64 2
    forecast_year       (forecast_time) int64 2000 2001 2002
Data variables:
    t2m                 (forecast_time, lead_time, latitude, longitude) float32 dask.array<chunksize=(1, 46, 121, 240), meta=np.ndarray>
    pr                  (forecast_time, lead_time, latitude, longitude) float32 dask.array<chunksize=(1, 46, 121, 240), meta=np.ndarray>
Attributes:
    source_dataset_name:  temperature daily from NOAA NCEP CPC: Climate Predi...
    source_hosting:       IRIDL
    source_url:           http://iridl.ldeo.columbia.edu/SOURCES/.NOAA/.NCEP/...
    created_by_software:  climetlab-s2s-ai-challenge
    created_by_script:    tools/observations/makefile

In [ ]:
len(obs.lead_time)

In [ ]:
46 / 7

In [ ]:
dataset

In [ ]:
sample = dataset.isel(forecast_time=slice(20))

In [ ]:
sample

In [ ]:
sample = sample.persist()

In [ ]:
some_computed = sample.isnull().sum(dim=['latitude', 'longitude', 'realization'])

In [ ]:
some_computed.sum(dim=['variable', 'forecast_time']).compute().x.plot()

In [ ]:
some_computed.isel(lead_time=[0, 1]).sum(dim='variable').compute()

In [ ]:
sample.sel(variable='sst').isel(lead_time=slice(1, None)).sum(dim=['forecast_time', 'lead_time', 'realization']).compute().x.plot()

## Check y

In [ ]:
target = xr.open_mfdataset(dataset_files, group='/y', concat_dim='forecast_time', preprocess=preprocess_one_example)

In [ ]:
target = target.compute()

In [ ]:
target.isnull().sum(dim=['category', 'forecast_time', 'lead_time', 'variable']).y.plot()